In [ ]:
from api_and_repo_link import link_me_up
import api_and_repo_link
from importlib import reload
reload(api_and_repo_link)
link_me_up()

In [ ]:

from MAPPINGS import COUNTRY_VALUE_MAP, STATUS_VALUE_MAP, STATUS_VALUE_MAP_CSV, SUBSIDIARY_VALUE_MAP, SUBSIDIARY_VALUE_MAP_CSV, CURRENCY_VALUE_MAP, COLUMN_MAPPING, COLUMN_MAPPING_ADDRESS_BOOK, COLUMN_MAPPING_ADDRESS

import pandas
from oliverbonas_source_dags.apis.api_netsuite import APIConfig_netsuite

In [ ]:
from multithread_uploader import upload_all_these_records
api = APIConfig_netsuite()
from util.get_ns_number import get_ns_number


In [31]:
Customer = api.client.get_type("ns13:Customer")
CustomerSearchBasic = api.client.get_type("ns5:CustomerSearchBasic")
get_ns_number(api, "CustomerSearchBasic")

failed for ns 0
failed for ns 1
failed for ns 2
failed for ns 3
failed for ns 4
CustomerSearchBasic = api.client.get_type("ns5:CustomerSearchBasic")


In [ ]:
def do_value_mappings(old_column_name, customer_dict):
    value = customer_dict[old_column_name]
    # this werid character \xa0 is removed if found...!
    if type(value) == str:
        value = value.replace("\xa0", " ")

    if old_column_name.endswith("_country"):
        return COUNTRY_VALUE_MAP[value]
    elif old_column_name == "currency":
        return CURRENCY_VALUE_MAP[value]
    elif old_column_name == "status":
        return STATUS_VALUE_MAP[STATUS_VALUE_MAP_CSV[value]]
    elif old_column_name == "subsidiary":
        return SUBSIDIARY_VALUE_MAP[SUBSIDIARY_VALUE_MAP_CSV[value]]
    else:
        return value

def to_dict(customer_series):
    customer_series[customer_series.isna()] = None # might work!
    customer_dict = customer_series.to_dict()

    # deal with value mappings here straight away
    for oldCol in customer_dict.keys():
        customer_dict[oldCol] = do_value_mappings(oldCol, customer_dict)

    # now deal with the column mappings
    for oldCol, newCol in COLUMN_MAPPING.items():
        customer_dict[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    addressbookList = {
        "addressbook": [
            {
                "addressbookAddress": dict(),
            },
            {
                "addressbookAddress": dict(),
            },
        ]
    }

    Addbook1 = addressbookList["addressbook"][0]
    Addbook2 = addressbookList["addressbook"][1]
    Address1 = addressbookList["addressbook"][0]["addressbookAddress"]
    Address2 = addressbookList["addressbook"][1]["addressbookAddress"]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS_BOOK.items():
        Addbook = Addbook1 if oldCol.startswith("Address1_") else Addbook2

        Addbook[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS.items():
        Address = Address1 if oldCol.startswith("Address1_") else Address2

        Address[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    customer_dict["addressbookList"] = addressbookList
    
    customer_dict["entityId"] = "customer_human_" + str(customer_dict["entityId"])
    customer_dict["externalId"] = "customer_human_" + str(customer_dict["externalId"])

    return customer_dict

In [33]:
response = api.service_proxy.search(
    CustomerSearchBasic(),
    _soapheaders = api.build_soap_headers()
)
custs = response.body.searchResult.recordList.record

AttributeError: 'NoneType' object has no attribute 'addressbook'

In [ ]:
test_df = pandas.read_csv("test_input_dummy_customers.csv")

test_list = [Customer(**to_dict(row)) for index, row in test_df.iterrows()]
test_list

In [26]:
import os
real_df = pandas.concat(
    [
        pandas.read_csv(f"sense/{fname}")
        for fname in os.listdir("sense")
    ]
)

In [19]:
trans_df = pandas.concat(
    [pandas.read_csv("../transaction_uploader/SLS_NS_20220101_20230430_1.csv"),
     pandas.read_csv("../transaction_uploader/SLS_NS_20220101_20230430_2.csv"),
     ]
)
customers = trans_df[trans_df.Customer.notna()].Customer.astype(int).astype(str)
customers

C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\IPython\core\interactiveshell.py:3284 DtypeWarning: Columns (0,1,3,4,7,13,14,15,16,18,19,20,21,24,25,26,27,28,31,32,33,34,35,38) have mixed types.Specify dtype option on import or set low_memory=False.

0        1512157
1        1512157
2        1654460
3         119057
4         119057
          ...   
23970     869291
23971    1498546
23972    1498546
23973    1521293
23974    1521293
Name: Customer, Length: 22408, dtype: object

In [25]:
customers.astype(str).str.len().value_counts()

7    12540
6     8100
5     1551
4      178
2       20
3       19
Name: Customer, dtype: int64

In [42]:
cust_df = real_df[real_df.entityId.isin(customers)]

,externalId,entityId,isPerson,companyName,firstName,lastName,subsidiary,email,status,currency,...,Address2_AddressName,Address2_attention,Address2_phone,Address2_line1,Address2_city,Address2_state,Address2_zipCode,Address2_country,Address2_defaultBillTo,Address2_defaultShipTo
0,10000001,10000001,True,NaN,Priyani,Peters,Parent Company,priyanip@hotmail.co.uk,CustomER-Closed Won,GBP,...,Ship-To,Priyani Peters,7788115363,4 St. Helens Crescent\nSandhurst,Sandhurst,Berkshire,GU47 9AX,United Kingdom,False,True
1,10000002,10000002,True,NaN,Anna,Tippett,Parent Company,atippett2016@gmail.com,CustomER-Closed Won,GBP,...,Ship-To,Anna Tippett,7734937388,60 Pen Y Fro\nDunvant,Swansea,Abertawe,SA2 7TP,United Kingdom,False,True
2,10000005,10000005,True,NaN,Natalie,Hamilton-Martin,Parent Company,nataliehm@icloud.com,CustomER-Closed Won,GBP,...,Ship-To,Natalie Hamilton-Martin,7828725615,20 Ashley Road\nAshley Cottage,Thames Ditton,Surrey,KT7 0NJ,United Kingdom,False,True
3,10000006,10000006,True,NaN,Juliet,Smith,Parent Company,juliet.67day@sky.com,CustomER-Closed Won,GBP,...,Ship-To,Juliet Smith,1206263907,8 The Blundens\nStoke By Nayland,Colchester,Suffolk,CO6 4RH,United Kingdom,False,True
4,10000009,10000009,True,NaN,Laura,Shepherd,Parent Company,huckers16@hotmail.com,CustomER-Closed Won,GBP,...,Ship-To,Laura Shepherd,7919576290,15 Nursery Close,Biggleswade,Bedfordshire,SG18 0HR,United Kingdom,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,1436275,1436275,True,NaN,Layla,Painter,Parent Company,laylapainter@hotmail.com,CustomER-Closed Won,GBP,...,Ship-To,Layla Painter,7818028152,Ivy Cottage\nArthington Lane Arthington,Otley,West Yorkshire,LS21 1PG,United Kingdom,False,True
24995,1436284,1436284,True,NaN,Alexandra,Drewnicki,Parent Company,alexandradrewnicki@hotmail.com,CustomER-Closed Won,GBP,...,Ship-To,Alexandra Drewnicki,7902665076,Beau Lodge\nLongbank,Bewdley,Worcestershire,DY12 2QS,United Kingdom,False,True
24996,1436285,1436285,True,NaN,Sarah,Marjoram,Parent Company,sarah.marjoram@ntlworld.com,CustomER-Closed Won,GBP,...,Ship-To,Sarah Marjoram,7787552622,75 Davies Road\nWest Bridgford,Nottingham,Nottingham,NG2 5JB,United Kingdom,False,True
24997,1436298,1436298,True,NaN,Kalpana,Fitzpatrick,Parent Company,kalpz@hotmail.com,CustomER-Closed Won,GBP,...,Ship-To,Kalpana Fitzpatrick,-,Flat 3\nAshdown Court,London,NaN,SE22 8NP,United Kingdom,False,True


In [44]:
lads = [Customer(**to_dict(row)) for index, row in real_df.iterrows()]

In [58]:
for pos, lad in enumerate(lads):
    lad.externalId = lad.externalId[15:]
    lad.entityId = lad.entityId[15:]
    if pos % 1000 == 0:
        print("done " + str(pos))

done 0
done 1000
done 2000
done 3000
done 4000
done 5000
done 6000
done 7000
done 8000
done 9000
done 10000
done 11000
done 12000
done 13000
done 14000
done 15000
done 16000
done 17000
done 18000
done 19000
done 20000
done 21000
done 22000
done 23000
done 24000
done 25000
done 26000
done 27000
done 28000
done 29000
done 30000
done 31000
done 32000
done 33000
done 34000
done 35000
done 36000
done 37000
done 38000
done 39000
done 40000
done 41000
done 42000
done 43000
done 44000
done 45000
done 46000
done 47000
done 48000
done 49000
done 50000
done 51000
done 52000
done 53000
done 54000
done 55000
done 56000
done 57000
done 58000
done 59000
done 60000
done 61000
done 62000
done 63000
done 64000
done 65000
done 66000
done 67000
done 68000
done 69000
done 70000
done 71000
done 72000
done 73000
done 74000
done 75000
done 76000
done 77000
done 78000
done 79000
done 80000
done 81000
done 82000
done 83000
done 84000
done 85000
done 86000
done 87000
done 88000
done 89000
done 90000
done 91000
d

In [71]:
upload_all_these_records(lads, api, "addList")

> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] records in this chunk
> [200] re

Exception in thread Thread-117:
Traceback (most recent call last):
  File "C:\Users\Thomas.fuller\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Thomas.fuller\AppData\Local\Programs\Python\Python39\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Thomas.fuller\NotDrive\Repos\netsuite_uploader\multithread_uploader.py", line 31, in soap_uploader_thread
    response = api.service_proxy.addList(
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\proxy.py", line 46, in __call__
    return self._proxy._binding.send(
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\wsdl\bindings\soap.py", line 123, in send
    envelope, http_headers = self._create(
  File "C:\Users\Thomas.fuller\NotDrive\VENV\simplyvenv\lib\site-packages\zeep\wsdl\bindings\soap.py", line 73, in _create
    serialized = operation_obj.create(*args, *

# thread (19) total [423608] records [200]  success [200] failure [0]
6:12:13.224534
# thread (18) total [423808] records [200]  success [200] failure [0]
6:12:16.820214
# thread (7) total [424008] records [200]  success [200] failure [0]
6:12:24.603319
# thread (15) total [424208] records [200]  success [200] failure [0]
6:12:35.337159
# thread (9) total [424408] records [200]  success [200] failure [0]
6:12:41.611038
# thread (21) total [424608] records [200]  success [200] failure [0]
6:13:14.423437
# thread (8) total [424808] records [200]  success [200] failure [0]
6:13:17.745258
# thread (16) total [425008] records [200]  success [200] failure [0]
6:13:21.283166
# thread (20) total [425208] records [200]  success [200] failure [0]
6:13:22.908468
# thread (5) total [425408] records [200]  success [200] failure [0]
6:13:25.730239
# thread (13) total [425608] records [200]  success [200] failure [0]
6:13:27.061064
# thread (1) total [425808] records [200]  success [200] failure [0]


In [ ]:
import multithread_uploader
reload(multithread_uploader)
from multithread_uploader import upload_all_these_records
upload_all_these_records(test_list, api, "addList")

In [70]:
import os
pandas.concat(
    [
        pandas.read_csv("thread_logs/" + filename)
        for filename in os.listdir("thread_logs")
    ]
)

,key_0,afterSubmitFailed,code,message,num_status_details,success,type,accessRole,accountNumber,addressbookList,...,thirdPartyAcct,thirdPartyCountry,thirdPartyZipcode,title,unbilledOrders,url,vatRegNumber,visits,webLead,api_method
0,0,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
1,0,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
2,0,0,NaN,NaN,1,True,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
3,0,0,NaN,NaN,1,True,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
4,1,0,NaN,NaN,1,True,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,35,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
42,36,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
43,37,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
44,38,NaN,DUP_ENTITY,This entity already exists.,1,False,ERROR,NaN,NaN,{\n 'addressbook': [\n {\n ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,addList
